In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from navec import Navec
from slovnet.model.emb import NavecEmbedding
from tg.projects.retell.retell_utils.metrics import get_cosine_sim, get_jaccard_index,plot_bar_jac_cos_metric
from pathlib import Path
from tg.projects.retell.retell_utils.corpora_sugar import CorpusSugar
from tg.grammar_ru.corpus import CorpusReader
import pyarrow.parquet as pq

Пересказ

In [5]:
retell_corpus = CorpusReader(Path("C:/Users/nikya/PycharmProjects/grammar/grammar_ru/tg/projects/retell/featurized_corpuses/All retell.featurized.retell.corpus.zip"))
martin_mapping = retell_corpus.read_mapping_data()

martin_mapping

,id_to
b512c9bc-9ff9-46a1-bdb4-9b62736b1452,57af3a88-680a-43d4-a022-09bc23026182
e5ad3fe9-1e1d-4983-8177-77028423f228,baddd09c-556f-4cb6-a122-090a65b6fb86
38eb4f83-e4a2-405d-a37d-612bc38b957b,2d517c70-9c1c-46ed-8a6c-8bfbef138aaf
616b20f4-40c9-4d02-88db-182ff75bf9b1,3073ab42-6a53-4056-ab81-a44ebaf9639c
66fc9afb-ac6d-409c-bd09-7947fa18810d,02ea2a22-1727-45ed-b1a9-06f9c8230e3a
...,...
11b2f4c2-2139-44be-82b7-a219f8e549a6,4af9c962-e1af-411e-bd3b-c3383858043a
8cf0fb48-6a57-45b7-ad22-503830bf103f,bbcfdeee-4cc3-4d10-9a27-5775f76f3147
5d8e6bb0-e0e3-4f5d-b3d4-f01d4f0b54b1,ff972aa0-cc17-4040-bb46-cf76b39dcbf4
0f42012f-c408-422a-a595-0f40b7a56cfa,f48cc169-3d2d-43bf-8790-8acc4e953047


In [6]:
retell_corpus.get_toc() 


,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,headers,author,book_name,text_type,max_id
file_id,,,,,,,,,,,,
2803ce5f-d1ef-4aa8-826c-4b47a2bf7568,Роулинг-Гарри Поттер и Тайная комната-retell.md,2023-04-13 23:59:49.112629,0,212,1013,0,День рождения — хуже некуда,День рождения — хуже некуда,Роулинг,Гарри Поттер и Тайная комната,retell,213
6a139af8-979f-4dc9-ac9b-f4faeec8360b,Роулинг-Гарри Поттер и Тайная комната-retell.md,2023-04-13 23:59:49.117094,1,539,2479,1,Добби предупреждает об опасности,Добби предупреждает об опасности,Роулинг,Гарри Поттер и Тайная комната,retell,10753
005cab1f-60cd-4716-bae9-5e695609f72e,Роулинг-Гарри Поттер и Тайная комната-retell.md,2023-04-13 23:59:49.136059,2,484,2154,2,Нора (глава),Нора (глава),Роулинг,Гарри Поттер и Тайная комната,retell,21238
1d7074c1-7daa-4e37-b5fe-3b1e8454a332,Роулинг-Гарри Поттер и Тайная комната-retell.md,2023-04-13 23:59:49.154647,3,546,2685,3,Флориш и Блоттс (глава),Флориш и Блоттс (глава),Роулинг,Гарри Поттер и Тайная комната,retell,31785
31b596ee-2c93-4172-89ac-c6944141fdb3,Роулинг-Гарри Поттер и Тайная комната-retell.md,2023-04-13 23:59:49.173768,4,513,2439,4,Гремучая ива (глава),Гремучая ива (глава),Роулинг,Гарри Поттер и Тайная комната,retell,42299
...,...,...,...,...,...,...,...,...,...,...,...,...
6bdf3842-e93f-4696-8654-6f168e9e1f43,Мартин-Пир стервятников-retell.md,2023-04-14 00:00:23.930643,41,288,1504,1139,"Пир_стервятников,_Алейна_II","Пир_стервятников,_Алейна_II",Мартин,Пир стервятников,retell,11653796
77dbafba-7725-4903-ad84-6421a4df282c,Мартин-Пир стервятников-retell.md,2023-04-14 00:00:23.948444,42,793,3483,1140,"Пир_стервятников,_Бриенна_VIII","Пир_стервятников,_Бриенна_VIII",Мартин,Пир стервятников,retell,11664590
06c7b201-6f75-4da5-9b43-bf660c847237,Мартин-Пир стервятников-retell.md,2023-04-14 00:00:23.967865,43,778,3534,1141,"Пир_стервятников,_Серсея_X","Пир_стервятников,_Серсея_X",Мартин,Пир стервятников,retell,11675369


In [10]:
author_name = "Мартин"
retell_author_df = retell_corpus.get_toc() 

all_retell_author_df = retell_author_df[retell_author_df['author'] == author_name]

all_retell_author_df['book_name'].unique() 


array(['Пир стервятников', 'Игра престолов (роман)', 'Битва королей',
       'Буря мечей', 'Танец с драконами'], dtype=object)

In [2]:
retell_author_df = all_retell_author_df[all_retell_author_df.text_type == 'retell' and all_retell_author_df.book_name == 'Буря мечей']


NameError: name 'all_retell_author_df' is not defined

In [32]:
retell_books = [retell_author_df[retell_author_df['book_name'] == name_of_book] for name_of_book in
                        retell_author_df['book_name'].unique()]

retell_books

[                                                         filename  \
 file_id                                                             
 b512c9bc-9ff9-46a1-bdb4-9b62736b1452  Мартин-Буря мечей-retell.md   
 e5ad3fe9-1e1d-4983-8177-77028423f228  Мартин-Буря мечей-retell.md   
 38eb4f83-e4a2-405d-a37d-612bc38b957b  Мартин-Буря мечей-retell.md   
 616b20f4-40c9-4d02-88db-182ff75bf9b1  Мартин-Буря мечей-retell.md   
 66fc9afb-ac6d-409c-bd09-7947fa18810d  Мартин-Буря мечей-retell.md   
 ...                                                           ...   
 f1f9792f-8a5d-4814-b4d0-df042881c778  Мартин-Буря мечей-retell.md   
 7544880d-a524-47df-a27c-b0c28d159649  Мартин-Буря мечей-retell.md   
 e74e221a-3fb3-465b-a01e-395033ce5464  Мартин-Буря мечей-retell.md   
 54a22b9e-0b3f-460f-92db-53f936549987  Мартин-Буря мечей-retell.md   
 e7158e99-568c-4d96-8ed4-54407415823c  Мартин-Буря мечей-retell.md   
 
                                                       timestamp  part_index  \
 file_id

In [33]:
books = []
mapping_retell_to_text = retell_corpus.read_mapping_data()
print(mapping_retell_to_text)

for ret_book in retell_books:
    text_book_indx = mapping_retell_to_text.loc[ret_book.index].index
    print(text_book_indx)
    text_book_chapters = retell_author_df.loc[text_book_indx]
    books.append(text_book_chapters)

print(books)
    


                                                                     id_to
b512c9bc-9ff9-46a1-bdb4-9b62736b1452  57af3a88-680a-43d4-a022-09bc23026182
e5ad3fe9-1e1d-4983-8177-77028423f228  baddd09c-556f-4cb6-a122-090a65b6fb86
38eb4f83-e4a2-405d-a37d-612bc38b957b  2d517c70-9c1c-46ed-8a6c-8bfbef138aaf
616b20f4-40c9-4d02-88db-182ff75bf9b1  3073ab42-6a53-4056-ab81-a44ebaf9639c
66fc9afb-ac6d-409c-bd09-7947fa18810d  02ea2a22-1727-45ed-b1a9-06f9c8230e3a
...                                                                    ...
11b2f4c2-2139-44be-82b7-a219f8e549a6  4af9c962-e1af-411e-bd3b-c3383858043a
8cf0fb48-6a57-45b7-ad22-503830bf103f  bbcfdeee-4cc3-4d10-9a27-5775f76f3147
5d8e6bb0-e0e3-4f5d-b3d4-f01d4f0b54b1  ff972aa0-cc17-4040-bb46-cf76b39dcbf4
0f42012f-c408-422a-a595-0f40b7a56cfa  f48cc169-3d2d-43bf-8790-8acc4e953047
70d19c07-74e8-4165-9f0c-8309253170a9  c7cd19f2-7045-45e0-ad23-419119765ef6

[687 rows x 1 columns]
Index(['b512c9bc-9ff9-46a1-bdb4-9b62736b1452',
       'e5ad3fe9-1e1d-4983-81

In [34]:
def get_sentences(chapter,corpus):
    chptr = corpus.get_bundles([chapter]).single()
    chapter_id = chptr.src
    sentences_id = np.array(chapter_id['sentence_id'].unique())
    sentences = [chapter_id['word'].values[chapter_id['sentence_id'] == sentence_id]
                 for sentence_id in sentences_id]
    return sentences

In [35]:
for book in books[:1]:
    extract_retell = []
    for chapter in book.index:
        sentences = get_sentences(chapter,retell_corpus)
        for sentence in sentences:
            s = ''
            for word in sentence:
                s += word
                s += " "
                

            

пересказ текста на англиском

In [57]:
rdf_test = pd.read_parquet("mapping.parquet")

rdf_test

,id_to
b512c9bc-9ff9-46a1-bdb4-9b62736b1452,57af3a88-680a-43d4-a022-09bc23026182
e5ad3fe9-1e1d-4983-8177-77028423f228,baddd09c-556f-4cb6-a122-090a65b6fb86
38eb4f83-e4a2-405d-a37d-612bc38b957b,2d517c70-9c1c-46ed-8a6c-8bfbef138aaf
616b20f4-40c9-4d02-88db-182ff75bf9b1,3073ab42-6a53-4056-ab81-a44ebaf9639c
66fc9afb-ac6d-409c-bd09-7947fa18810d,02ea2a22-1727-45ed-b1a9-06f9c8230e3a
...,...
11b2f4c2-2139-44be-82b7-a219f8e549a6,4af9c962-e1af-411e-bd3b-c3383858043a
8cf0fb48-6a57-45b7-ad22-503830bf103f,bbcfdeee-4cc3-4d10-9a27-5775f76f3147
5d8e6bb0-e0e3-4f5d-b3d4-f01d4f0b54b1,ff972aa0-cc17-4040-bb46-cf76b39dcbf4
0f42012f-c408-422a-a595-0f40b7a56cfa,f48cc169-3d2d-43bf-8790-8acc4e953047


In [19]:
retell_eng_corpus = CorpusReader(Path("C:/Users/nikya/PycharmProjects/grammar/grammar_ru/tg/projects/retell/featurized_corpuses/eng/Martin.featurized.retell.corpus.zip"))

martin_eng_mapping = retell_eng_corpus.read_mapping_data()

martin_eng_mapping


,book_chapter_id
aebf9cd8-770e-4f4c-9778-77166d4ee87d,1104469b-e276-4909-a3ad-3ae962833bba
c67d8477-4a35-4c0c-a5f1-a1cd61ed7900,cd431847-3453-403e-99ab-a5c08b1b02f7
5d191b46-a50c-4ebd-bc9e-4cb4d801992b,c8deee67-37a4-4b71-a004-8a8d83fd24b6
6c8fc056-db82-4c14-82f3-c56a03d560ec,35b83cee-e4a1-4b51-b891-b63883f2dbd4
99ef0c8c-d5c2-457c-87c0-0a3c68293e3c,10cf20b8-35c3-4e8a-a1d7-4406d035ad40
...,...
9aa00f14-54ec-44c6-865f-73bba133f8a8,f336ab7d-78ae-454a-aad8-6435b2e1c421
b5ee9c31-b294-45ce-a455-724fa9b35e8b,afce8beb-4754-4eb0-bd9e-a70217e33363
b51bb5e2-da83-4f43-ad11-6d9bbf92e2a5,cff38c73-d81a-45c6-9bae-2085d7e43c51
687a2160-08d6-4abf-8365-19f43d85620c,dcc9df0f-880a-485d-97a7-3a07689c8a13


In [18]:
retell_eng_corpus.get_toc() 

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,headers,author,book_name,text_type,max_id
file_id,,,,,,,,,,,,
aebf9cd8-770e-4f4c-9778-77166d4ee87d,Martin-A Clash of Kings-retell.md,2023-08-10 22:21:29.332969,0,3536,14150,0,Prologue-Chapter_0,Prologue-Chapter_0,Martin,A Clash of Kings,retell,3537
c67d8477-4a35-4c0c-a5f1-a1cd61ed7900,Martin-A Clash of Kings-retell.md,2023-08-10 22:21:29.356369,1,1201,4561,1,Arya_I-Chapter_1,Arya_I-Chapter_1,Martin,A Clash of Kings,retell,14739
5d191b46-a50c-4ebd-bc9e-4cb4d801992b,Martin-A Clash of Kings-retell.md,2023-08-10 22:21:29.380943,2,2031,8290,2,Sansa_I-Chapter_2,Sansa_I-Chapter_2,Martin,A Clash of Kings,retell,26771
6c8fc056-db82-4c14-82f3-c56a03d560ec,Martin-A Clash of Kings-retell.md,2023-08-10 22:21:29.417667,3,2221,8933,3,Tyrion_I-Chapter_3,Tyrion_I-Chapter_3,Martin,A Clash of Kings,retell,38993
99ef0c8c-d5c2-457c-87c0-0a3c68293e3c,Martin-A Clash of Kings-retell.md,2023-08-10 22:21:29.445902,4,2380,9242,4,Bran_I-Chapter_4,Bran_I-Chapter_4,Martin,A Clash of Kings,retell,51374
...,...,...,...,...,...,...,...,...,...,...,...,...
9aa00f14-54ec-44c6-865f-73bba133f8a8,Martin-A Dance with Dragons-retell.md,2023-08-10 22:21:42.145730,68,179,702,339,The_Dragontamer-Chapter_68,The_Dragontamer-Chapter_68,Martin,A Dance with Dragons,retell,3655455
b5ee9c31-b294-45ce-a455-724fa9b35e8b,Martin-A Dance with Dragons-retell.md,2023-08-10 22:21:42.162876,69,367,1416,340,Jon_XIII-Chapter_69,Jon_XIII-Chapter_69,Martin,A Dance with Dragons,retell,3665823
b51bb5e2-da83-4f43-ad11-6d9bbf92e2a5,Martin-A Dance with Dragons-retell.md,2023-08-10 22:21:42.180743,70,999,4143,341,The_Queen's_Hand-Chapter_70,The_Queen's_Hand-Chapter_70,Martin,A Dance with Dragons,retell,3676823


In [39]:
retell_author_df = retell_eng_corpus.get_toc() 

retell_books = [retell_author_df[retell_author_df['book_name'] == name_of_book] for name_of_book in
                        retell_author_df['book_name'].unique()]

retell_books

[                                                               filename  \
 file_id                                                                   
 aebf9cd8-770e-4f4c-9778-77166d4ee87d  Martin-A Clash of Kings-retell.md   
 c67d8477-4a35-4c0c-a5f1-a1cd61ed7900  Martin-A Clash of Kings-retell.md   
 5d191b46-a50c-4ebd-bc9e-4cb4d801992b  Martin-A Clash of Kings-retell.md   
 6c8fc056-db82-4c14-82f3-c56a03d560ec  Martin-A Clash of Kings-retell.md   
 99ef0c8c-d5c2-457c-87c0-0a3c68293e3c  Martin-A Clash of Kings-retell.md   
 ...                                                                 ...   
 8a7f4dec-8e72-4711-8262-0332fb0c77fe  Martin-A Clash of Kings-retell.md   
 8f8b0287-c262-4b41-a8de-ba69b776ed2c  Martin-A Clash of Kings-retell.md   
 7b5212a3-1242-49d5-bb54-d3750b064475  Martin-A Clash of Kings-retell.md   
 bfb6d085-6abd-4b6e-937a-39034d6a29a1  Martin-A Clash of Kings-retell.md   
 0194bf07-baf8-485f-aaf8-2b9d7de51e49  Martin-A Clash of Kings-retell.md   
 
          

In [40]:
books = []
mapping_retell_to_text = retell_eng_corpus.read_mapping_data()
print(mapping_retell_to_text)

for ret_book in retell_books:
    text_book_indx = mapping_retell_to_text.loc[ret_book.index].index
    print(text_book_indx)
    text_book_chapters = retell_author_df.loc[text_book_indx]
    books.append(text_book_chapters)

print(books)

                                                           book_chapter_id
aebf9cd8-770e-4f4c-9778-77166d4ee87d  1104469b-e276-4909-a3ad-3ae962833bba
c67d8477-4a35-4c0c-a5f1-a1cd61ed7900  cd431847-3453-403e-99ab-a5c08b1b02f7
5d191b46-a50c-4ebd-bc9e-4cb4d801992b  c8deee67-37a4-4b71-a004-8a8d83fd24b6
6c8fc056-db82-4c14-82f3-c56a03d560ec  35b83cee-e4a1-4b51-b891-b63883f2dbd4
99ef0c8c-d5c2-457c-87c0-0a3c68293e3c  10cf20b8-35c3-4e8a-a1d7-4406d035ad40
...                                                                    ...
9aa00f14-54ec-44c6-865f-73bba133f8a8  f336ab7d-78ae-454a-aad8-6435b2e1c421
b5ee9c31-b294-45ce-a455-724fa9b35e8b  afce8beb-4754-4eb0-bd9e-a70217e33363
b51bb5e2-da83-4f43-ad11-6d9bbf92e2a5  cff38c73-d81a-45c6-9bae-2085d7e43c51
687a2160-08d6-4abf-8365-19f43d85620c  dcc9df0f-880a-485d-97a7-3a07689c8a13
73263dfb-f207-4d6a-a0b8-58465d47688b  c23a1119-c5b9-4e0f-8d2c-67e2ab89a890

[344 rows x 1 columns]
Index(['aebf9cd8-770e-4f4c-9778-77166d4ee87d',
       'c67d8477-4a35-4c0c-a5

In [55]:
from googletrans import Translator

for book in books[:1]:
    extract_retell = []
    for chapter in book.index:
        sentences = get_sentences(chapter,retell_eng_corpus)
        for sentence in sentences:
            translator = Translator()
            s = ''
            for word in sentence:
                s += word
                s += " "
            result = translator.translate("hello world", src='en', dest='ru')

            print(result.text) 


            



                
                

AttributeError: 'NoneType' object has no attribute 'group'